## Notebook Overview

This notebook builds upon the insights gained from the previous notebook (`03_exploratory_data_analysis.ipynb`) and focuses on **Feature Engineering**. This stage involves creating new features and transforming existing ones based on our statistical findings and domain knowledge to enhance our credit risk model.

### 0.4.1 Objectives

The main objectives of this notebook are:

1. **Leverage EDA Insights:** Utilize the patterns and relationships uncovered in our exploratory data analysis to guide our feature engineering efforts.
2. **Create Non-Linear Transformations:** Develop binned versions of continuous variables to capture non-linear relationships with the target variable.
3. **Implement Domain-Specific Features:** Create new features based on financial domain knowledge that are known to be relevant in credit risk assessment.
4. **Enhance Model Input:** Generate features that can potentially improve our model's predictive power and interpretability.
5. **Prepare for Modeling:** Finalize the feature set that will be used in our subsequent modeling efforts.

### 0.4.2 Importance of Feature Engineering

Feature engineering plays a crucial role in improving machine learning model performance:

- **Capture Complex Relationships:** Engineered features can help models capture non-linear and intricate relationships that may not be apparent in the raw data.
- **Incorporate Domain Knowledge:** It allows us to inject domain expertise into our data, potentially improving model performance and interpretability.
- **Improve Model Generalization:** Well-engineered features can help models generalize better to unseen data.
- **Enhance Interpretability:** Carefully crafted features can make model predictions more understandable and actionable for stakeholders.

### 0.4.3 Our Approach

In this notebook, we will focus on the following feature engineering tasks:

1. **Age Binning:** Create age groups (18-25, 26-35, 36-45, 46-55, 56-65, 65+) to capture non-linear relationships with default risk.
2. **Financial Ratios:** Implement key financial ratios including:
   - Debt-to-income ratio
   - Credit-to-goods price ratio
   - Annuity-to-income ratio
3. **Stability Indicators:** Derive new features such as:
   - Employed-to-age ratio
   - Flag for when credit amount exceeds goods price
4. **Credit Score Aggregation:** Generate an average of external source scores for a more stable overall credit score indicator.
5. **Income and Credit Amount Binning:** Create bins for income and credit amount variables to identify potential threshold effects and improve model robustness to outliers.

By the end of this notebook, we will have a rich set of engineered features that leverage both our data-driven insights and domain knowledge, setting a strong foundation for our subsequent modeling efforts.